In [14]:
import torch, os
import pandas as pd
import torch.nn as nn

In [2]:


def which_mat(TS, n,p = 0):
    
    return TS.iloc()


class TS_based_picker(nn.modules):
    def __init__(self, kernel = , in_channels =4,  hidden_size = 128):
        """
        
        
        """
        super(TS_based_picker, self).__init__(self):
        pads  == kernel//2
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel)
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d()
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size = (kenrnel-2), padding = (pads-1) )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d()
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size = (kenrnel-2), padding = (pads-1))
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d()
            )
        
        self.
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data
        self.cater = torch.cat((self.c23, self.c3), dim=1)
        
        #
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-2-eaa2c12dd2d4>, line 1)

In [8]:
b = pd.read_csv("100stocks.csv")

In [15]:
def get_file(fname,
             origin,
             untar=False,
             extract=False,
             archive_format='auto',
             cache_dir='data'):
    datadir = os.path.join(cache_dir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    if untar:
        untar_fpath = os.path.join(datadir, fname)
        fpath = untar_fpath + '.tar.gz'
    else:
        fpath = os.path.join(datadir, fname)
    
    print(fpath)
    if not os.path.exists(fpath):
        print('Downloading data from', origin)

        error_msg = 'URL fetch failure on {}: {} -- {}'
        try:
            try:
                urlretrieve(origin, fpath)
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason))
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg))
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(fpath):
                os.remove(fpath)
            raise

    if untar:
        if not os.path.exists(untar_fpath):
            print('Extracting file.')
            with tarfile.open(fpath) as archive:
                archive.extractall(datadir)
        return untar_fpath

    if extract:
        _extract_archive(fpath, datadir, archive_format)

    return fpath

data_fpath = get_file(fname='SP500.pkl',
                      origin='/https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP500.pkl', 
                         untar=False)
data_fpath = get_file(fname='SP400.pkl', 
                         origin='/https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP400.pkl', 
                         untar=False)

data/SP500.pkl


NameError: name 'URLError' is not defined